In [44]:
import pandas as pd
from functools import reduce
from IPython.display import IFrame 

In [53]:
location ='https://raw.githubusercontent.com/Public-Policy-COVID/'
fileEmma = location+'Emma-li/main/emma_data.csv'
fileBryn = location + 'Bryn-Bandt-Law/main/bryn_clean_data.csv'
fileAyushi = location + 'Ayushi-Dhar/main/ayushi_data.csv'
fileNick = location + 'Nick-Maue/main/race_final.csv'
emma_data = pd.read_csv(fileEmma)
bryn_data = pd.read_csv(fileBryn)
ayushi_data = pd.read_csv(fileAyushi)
nick_data = pd.read_csv(fileNick)

In [54]:
del ayushi_data['Unnamed: 0']
del emma_data['Unnamed: 0']
del bryn_data['Unnamed: 0']

,County,State,total_population,white_total,black_total,aian_total,asian_total,nhopi_total,multiracial_total,white_total_pct,black_total_pct,aian_total_pct,asian_total_pct,nhopi_total_pct,multiracial_total_pct
0,Adams,WA,20151,18056,447,1109,287,11,241,89.60,2.22,5.50,1.42,0.05,1.20
1,Asotin,WA,22521,21087,144,356,184,58,692,93.63,0.64,1.58,0.82,0.26,3.07
2,Benton,WA,201803,181938,3379,2504,6850,473,6659,90.16,1.67,1.24,3.39,0.23,3.30
3,Chelan,WA,78421,73391,511,1497,929,193,1900,93.59,0.65,1.91,1.18,0.25,2.42
4,Clallam,WA,76011,65939,791,4243,1420,143,3475,86.75,1.04,5.58,1.87,0.19,4.57
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
128,Wallowa,OR,7208,6890,33,68,36,10,171,95.59,0.46,0.94,0.50,0.14,2.37
129,Wasco,OR,26682,24271,215,1003,300,191,702,90.96,0.81,3.76,1.12,0.72,2.63
130,Washington,OR,601592,479049,15194,6827,70417,3168,26937,79.63,2.53,1.13,11.71,0.53,4.48
131,Wheeler,OR,1332,1224,3,34,14,3,54,91.89,0.23,2.55,1.05,0.23,4.05


In [50]:
merge1 = pd.merge(left=ayushi_data, right=emma_data, left_on=['County','State'], right_on=['County','State'],how='right')
merge1[['Number_of_beds','Number_of_hospitals']] = merge1[['Number_of_beds','Number_of_hospitals']].fillna(0)

In [52]:
merge2 = pd.merge(left=merge1, right=bryn_data, left_on=['County','State'], right_on=['County','State'],how='inner')

In [55]:
merged_data = pd.merge(left=merge2, right=nick_data, left_on=['County','State'], right_on=['County','State'],how='inner')

In [56]:
merged_data

,Number_of_beds,Number_of_hospitals,State,County,Urban_Rural_Code,Deaths_COVID,Deaths_total,never,rarely,sometimes,...,aian_total,asian_total,nhopi_total,multiracial_total,white_total_pct,black_total_pct,aian_total_pct,asian_total_pct,nhopi_total_pct,multiracial_total_pct
0,3667.0,22.0,CA,Alameda,Large central metro,573,"10,908",0.019,0.008,0.055,...,17797,540391,15761,89419,49.28,11.03,1.06,32.33,0.94,5.35
1,0.0,0.0,CA,Alpine,NaN,0,0,0.025,0.085,0.088,...,290,18,0,50,67.94,0.35,25.69,1.59,0.00,4.43
2,52.0,1.0,CA,Amador,Noncore,31,415,0.045,0.013,0.099,...,925,665,116,1344,89.65,2.68,2.33,1.67,0.29,3.38
3,553.0,6.0,CA,Butte,Small metro,101,"2,313",0.015,0.043,0.111,...,5547,10984,628,10127,85.65,1.90,2.53,5.01,0.29,4.62
4,25.0,1.0,CA,Calaveras,Noncore,12,385,0.045,0.019,0.098,...,905,859,122,1769,90.95,1.08,1.97,1.87,0.27,3.85
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
128,0.0,0.0,WA,Wahkiakum,NaN,0,0,0.045,0.057,0.079,...,60,39,8,161,93.18,0.43,1.43,0.93,0.19,3.84
129,87.0,1.0,WA,Walla Walla,Small metro,38,651,0.083,0.022,0.061,...,852,1059,247,1808,91.44,2.18,1.37,1.70,0.40,2.91
130,253.0,1.0,WA,Whatcom,Small metro,63,"1,881",0.042,0.028,0.061,...,7282,10092,751,9603,86.46,1.23,3.23,4.48,0.33,4.26
131,50.0,2.0,WA,Whitman,Micropolitan,21,269,0.002,0.009,0.049,...,422,5270,117,2268,81.90,1.99,0.84,10.51,0.23,4.52


In [57]:
merged_data.to_csv('Merged_data.csv',header=True)